In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import re
import datetime


In [ ]:
dataframe=pd.read_csv("/content/drive/MyDrive/forsk/CDR_Anaylysis Project/raw_cdr_data.csv",header=None, low_memory=False)  #

1. No column name   
   ( header = None )
    column no -  actual name
   * 1            serviceProvider
   * 5            direction
   * 9            startTime
   * 13           EndTime
   * 120          userId
   * 180          twoStageDialingDigits
   * 146          relatedCallId
   * 147          relatedCallIdReason
   * 267          vpDialingfacResult
   * 312          locationType
   * 345          userAgent

In [ ]:
print(dataframe[5].unique())


['Originating' 'Terminating' nan]


As per client requirement:
replace originative with outgoing, terminating with incoming

In [ ]:
print(dataframe[267].unique()) #Voice Portal

[nan 'Success']


replace Success with voice portal

In [ ]:
print(dataframe[312].unique()) # Secondary Device

['Shared Call Appearance' 'Primary Device' nan
 '6102632279@voip.evolveip.net' '6102632279.3@voip.evolveip.net'
 '6102300185.2@voip.evolveip.net' '6102634233@voip.evolveip.net'
 'BroadWorks Anywhere' '+12167770642' '8474633614@voip.evolveip.net'
 '6102634118@voip.evolveip.net' '4843940092@voip.evolveip.net'
 '3306224638.3@voip.evolveip.net' '6102300215.uc.ucp@voip.evolveip.net'
 '6102634230@voip.evolveip.net' '5854217091.1@voip.evolveip.net'
 '4843940107.5@voip.evolveip.net' '4845880779@voip.evolveip.net'
 '6102320565@voip.evolveip.net' '6102321825@voip.evolveip.net'
 '6102634238@voip.evolveip.net' 'jthompson@eip.local'
 '2166060566@voip.evolveip.net' '6102321485@voip.evolveip.net'
 '4843940105@voip.evolveip.net' '4845880772@voip.evolveip.net'
 '4804092704@voip.evolveip.net']


replace Shared Call Appearance with Secondary Device

#replace_simple_with_Standard_terminology

In [ ]:
def replace_simple_with_Standard_terminology(dataframe):
  dataframe[5]=dataframe[5].replace('Originating','Outgoing')
  dataframe[5]=dataframe[5].replace('Terminating','Incoming')
  dataframe[267]=dataframe[267].replace('Success','Voice Portal')
  dataframe[312]=dataframe[312].replace('Shared Call Appearance','Secondary Device')
  return dataframe

In [ ]:
dataframe=replace_simple_with_Standard_terminology(dataframe)

In [ ]:
print(dataframe[5].unique())

['Outgoing' 'Incoming' nan]


In [ ]:
print(dataframe[267].unique())

[nan 'Voice Portal']


In [ ]:
print(dataframe[312].unique())

['Secondary Device' 'Primary Device' nan '6102632279@voip.evolveip.net'
 '6102632279.3@voip.evolveip.net' '6102300185.2@voip.evolveip.net'
 '6102634233@voip.evolveip.net' 'BroadWorks Anywhere' '+12167770642'
 '8474633614@voip.evolveip.net' '6102634118@voip.evolveip.net'
 '4843940092@voip.evolveip.net' '3306224638.3@voip.evolveip.net'
 '6102300215.uc.ucp@voip.evolveip.net' '6102634230@voip.evolveip.net'
 '5854217091.1@voip.evolveip.net' '4843940107.5@voip.evolveip.net'
 '4845880779@voip.evolveip.net' '6102320565@voip.evolveip.net'
 '6102321825@voip.evolveip.net' '6102634238@voip.evolveip.net'
 'jthompson@eip.local' '2166060566@voip.evolveip.net'
 '6102321485@voip.evolveip.net' '4843940105@voip.evolveip.net'
 '4845880772@voip.evolveip.net' '4804092704@voip.evolveip.net']


* Client Requirement :

in dataframe[312] all unwanted email ids to be replaced
# remove_Unwanted_data

In [ ]:
def remove_Unwanted_data(datacolumn):
  for index in range (len(datacolumn)):
    if(datacolumn[index]=='Secondary Device'or datacolumn[index]=='Primary Device'):
      continue
    else:
      datacolumn[index]=np.nan
  return datacolumn



In [ ]:
dataframe[312]=remove_Unwanted_data(dataframe[312].tolist())
print(dataframe[312].unique())

['Secondary Device' 'Primary Device' nan]


* Client Requirement 3
*dataframe[147]--all nan values need to be replaced 
check:
dataframe[312],dataframe[267] - if there is missing data or nan data in 147, then concatenate 312 and 267. If 312 is empty read data from 267 add it to 147 columns. if 312 is not empty read data from 312 to 147.

#combine_All_Serivices

In [ ]:
dataframe[147].sample(5)

5150    NaN
9017    NaN
2363    NaN
5168    NaN
3813    NaN
Name: 147, dtype: object

In [ ]:
def combine_All_Serivices(datacolumn147,datacolumn312,datacolumn267):
  for index in range(len(datacolumn147)):
    if datacolumn147[index] is np.nan:
      if (datacolumn312[index] is not np.nan ) and (datacolumn267[index] is not np.nan):
        datacolumn147[index]=str(datacolumn312[index])+" "+str(datacolumn267[index])
      elif( datacolumn312[index] is not np.nan):
        datacolumn147[index]=datacolumn312[index]
      else:
        datacolumn147[index]=datacolumn267[index]

    else:
      continue
    return datacolumn147

In [ ]:
dataframe[147].sample(5)

15573    NaN
15348    NaN
8849     NaN
7935     NaN
10339    NaN
Name: 147, dtype: object

In [ ]:
dataframe[147]=combine_All_Serivices(dataframe[147].tolist(), dataframe[312].tolist(), dataframe[267].tolist())

In [ ]:
dataframe[147].sample(5)

5843     NaN
4279     NaN
375      NaN
5739     NaN
12167    NaN
Name: 147, dtype: object

There are chances that column 267 may have null values and since we have replaced 147 column data by 267 column data there is high probability that even 147 column will hold null values..that need to be treated

*Client Requirement:
Date time is in the format "20190620032717.906" it needs to be converted to ['2019-06-20 03:27:18']  date-month-day hours-mins-secs 

# call_time_fetcher

In [ ]:
def call_time_fetcher(data):
  for index in range(len(data)):
    data[index]=str(data[index])
    if(data[index]!='nan'):
      year=data[index][:4]
      month=data[index][4:6]
      day=data[index][6:8]
      hours=data[index][8:10]
      minutes=data[index][10:12]
      seconds=str(round(float(data[index][12:])))  # convert seconds.milliseconds to float then round it to convert to int and then to string
      if(int(seconds)>=60):
        minutes=int(minutes)+1
        seconds=int(seconds)-60
      if(int(minutes)>=60):
        hours=int(hours)+1
        minutes=int(minutes)-60

      data[index]=f"{year}-{month}-{day} {hours}:{minutes}:{seconds}"
    else:
      data[index]=np.nan
  return data

In [ ]:
data=["20190620032717.906"]
result=call_time_fetcher(data)
print(result)

['2019-06-20 03:27:18']


#hourly_range

In [ ]:
import datetime
import re
import numpy as np

def hourly_range(data):
    # Time column data is passed as a list
    # 03:27:17 AM'
    for index in range(len(data)):
        data[index] = str(data[index])
        if data[index]!="nan":
            if re.search("PM", data[index]):
                print("data at index{} is {}".format(index,data[index]))
                time_data =  re.findall("\d+", data[index])
                print("timedata {}".format(time_data))
                if time_data[0] != "12":
                    time_data = int(time_data[0]) + 12
                else:
                    time_data = time_data[0]
                
            else:
                time_data =  re.findall("\d+", data[index])
                if int(time_data[0]) == "12":
                    time_data = f"0{int(time_data[0]) - 12}"
                else:
                    time_data = time_data[0]
                
                
            data[index] = f"{time_data}:00 - {time_data}:59"
        else:
            data[index] = np.nan
    return data


In [ ]:

data = ['3:27:17 AM', '1:28:19 PM', 'nan', '7:23:52 PM']
result = hourly_range(data)
print(result)
# ['3:00 - 3:59', '13:00 - 13:59', nan, '19:00 - 19:59']



data at index1 is 1:28:19 PM
timedata ['1', '28', '19']
data at index3 is 7:23:52 PM
timedata ['7', '23', '52']
['3:00 - 3:59', '13:00 - 13:59', nan, '19:00 - 19:59']


#weekly_range

In [ ]:
# weekly_range
# ['2019-06-20', '2019-06-21', 'nan', '2019-06-25']
# ['Thursday', 'Friday', nan, 'Tuesday']



import datetime
import re
import numpy as np

def weekly_range(data):
    # Date column data is passed as a list
    # '2019-06-20' 
    for index in range(len(data)):
        data[index] = str(data[index])
        if data[index] != "nan":
            year, month, day = [int(x) for x in data[index].split("-")]
            result = datetime.date(year, month, day)
            data[index] = result.strftime("%A") # Full weekday name.
        else:
            data[index] = np.nan
    return data

data = ['2019-06-20', '2019-06-21', 'nan', '2019-06-25']
result = weekly_range(data)
print(result)
# ['Thursday', 'Friday', nan, 'Tuesday']

['Thursday', 'Friday', nan, 'Tuesday']


In [ ]:
raw_cdr_data=pd.read_csv( '/content/drive/MyDrive/forsk/CDR_Anaylysis Project/raw_cdr_data.csv',header=None,low_memory=False) 


In [ ]:
print(raw_cdr_data[9].tolist())

[20190620032717.906, 20190620032717.906, 20190620052652.523, 20190620052735.207, 20190620060434.773, 20190620060434.773, 20190620060452.785, 20190620060452.785, 20190620062055.11, 20190620062055.11, 20190620062521.12, 20190620062521.12, 20190620062558.51, 20190620062558.51, 20190620062616.52, 20190620062616.52, 20190620065126.387, 20190620065126.387, 20190620073226.656, 20190620073226.656, 20190620073740.965, 20190620073740.965, 20190620075108.805, 20190620075108.805, 20190620075126.81, 20190620075126.81, 20190620075306.52, 20190620075306.52, 20190620085700.92, 20190620085718.926, 20190620085718.926, 20190620085700.92, 20190620085726.44, 20190620085726.44, 20190620090524.426, 20190620090524.426, 20190620100903.992, 20190620100903.992, 20190620100922.0, 20190620100922.0, 20190620100927.64, 20190620100927.64, 20190620100945.6, 20190620100925.977, 20190620101313.668, 20190620101317.984, 20190620101317.984, 20190620101334.28, 20190620101334.28, 20190620100957.92, 20190620100957.92, 2019062

#datetime_divider

In [ ]:

#  Function to seperate the date and time from column 9
def datetime_divider(data):
    # data type of data is list
    # ["20190620032717.906"]
    for index in range(len(data)):    
        # find the digit if at begining only not others
        if re.match("^\d", str(data[index])): 
            regex = re.compile("\d{1,8}") 
            a = regex.findall(str(data[index]))
            data[index] = [a[0], a[1]]
        else:
            data[index] = [np.nan, np.nan]
        
    return data


#date_modifier

In [ ]:
# Function to convert the data in desired date format
def date_modifier(data):
    # data type of data is list
    # 20190620 should be converted to 2019-06-20
    for index in range(len(data)):
        if re.match("^\d", str(data[index])):
            year = str(data[index][:4])
            month = str(data[index][4:6])
            day = str(data[index][6:])
            data[index] = "-".join([year, month, day])
        else:
            data[index] =  np.nan
    return data


In [ ]:
raw_cdr_data=pd.read_csv("/content/drive/MyDrive/forsk/CDR_Anaylysis Project/raw_cdr_data.csv",header=None, low_memory=False)  #

In [ ]:
print(raw_cdr_data[9].tolist())

[20190620032717.906, 20190620032717.906, 20190620052652.523, 20190620052735.207, 20190620060434.773, 20190620060434.773, 20190620060452.785, 20190620060452.785, 20190620062055.11, 20190620062055.11, 20190620062521.12, 20190620062521.12, 20190620062558.51, 20190620062558.51, 20190620062616.52, 20190620062616.52, 20190620065126.387, 20190620065126.387, 20190620073226.656, 20190620073226.656, 20190620073740.965, 20190620073740.965, 20190620075108.805, 20190620075108.805, 20190620075126.81, 20190620075126.81, 20190620075306.52, 20190620075306.52, 20190620085700.92, 20190620085718.926, 20190620085718.926, 20190620085700.92, 20190620085726.44, 20190620085726.44, 20190620090524.426, 20190620090524.426, 20190620100903.992, 20190620100903.992, 20190620100922.0, 20190620100922.0, 20190620100927.64, 20190620100927.64, 20190620100945.6, 20190620100925.977, 20190620101313.668, 20190620101317.984, 20190620101317.984, 20190620101334.28, 20190620101334.28, 20190620100957.92, 20190620100957.92, 2019062

In [ ]:
raw_cdr_data['date'],raw_cdr_data['time']=zip(*datetime_divider(raw_cdr_data[9].tolist()))

In [ ]:
print(raw_cdr_data['date'])

0        20190620
1        20190620
2        20190620
3        20190620
4        20190620
           ...   
16733    20190625
16734    20190625
16735    20190625
16736    20190625
16737    20190625
Name: date, Length: 16738, dtype: object


In [ ]:
print(raw_cdr_data['time'])

0        032717
1        032717
2        052652
3        052735
4        060434
          ...  
16733    192334
16734    192352
16735    192352
16736    192405
16737    192143
Name: time, Length: 16738, dtype: object


In [ ]:
print(raw_cdr_data[312])

0        Shared Call Appearance
1                Primary Device
2                Primary Device
3                Primary Device
4        Shared Call Appearance
                  ...          
16733                       NaN
16734                       NaN
16735                       NaN
16736            Primary Device
16737    Shared Call Appearance
Name: 312, Length: 16738, dtype: object


In [ ]:
#change shared appearance to secondary device

raw_cdr_data=replace_simple_with_Standard_terminology (raw_cdr_data)

In [ ]:
print(raw_cdr_data[312])

0        Secondary Device
1          Primary Device
2          Primary Device
3          Primary Device
4        Secondary Device
               ...       
16733                 NaN
16734                 NaN
16735                 NaN
16736      Primary Device
16737    Secondary Device
Name: 312, Length: 16738, dtype: object


In [ ]:
print(raw_cdr_data[5])

0        Outgoing
1        Incoming
2        Outgoing
3        Incoming
4        Outgoing
           ...   
16733    Incoming
16734    Outgoing
16735    Incoming
16736    Incoming
16737    Incoming
Name: 5, Length: 16738, dtype: object


In [ ]:
raw_cdr_data[312]=remove_Unwanted_data(raw_cdr_data[312].tolist())

In [ ]:
#removed all email ids
print(raw_cdr_data[312])

0        Secondary Device
1          Primary Device
2          Primary Device
3          Primary Device
4        Secondary Device
               ...       
16733                 NaN
16734                 NaN
16735                 NaN
16736      Primary Device
16737    Secondary Device
Name: 312, Length: 16738, dtype: object


In [ ]:
raw_cdr_data[147]=combine_All_Serivices(raw_cdr_data[147].tolist(),raw_cdr_data[312].tolist(),raw_cdr_data[267].tolist())

In [ ]:
print(raw_cdr_data[147])

0              Secondary Device
1                Primary Device
2                           NaN
3                           NaN
4                           NaN
                  ...          
16733                       NaN
16734    Call Forward No Answer
16735                       NaN
16736                       NaN
16737                       NaN
Name: 147, Length: 16738, dtype: object


In [ ]:
#calculate duration of call
raw_cdr_data['starttime']=pd.to_datetime(call_time_fetcher(raw_cdr_data[9].tolist()))

In [ ]:
print(raw_cdr_data['starttime'])

0       2019-06-20 03:27:18
1       2019-06-20 03:27:18
2       2019-06-20 05:26:53
3       2019-06-20 05:27:35
4       2019-06-20 06:04:35
                ...        
16733   2019-06-25 19:23:34
16734   2019-06-25 19:23:52
16735   2019-06-25 19:23:52
16736   2019-06-25 19:24:06
16737   2019-06-25 19:21:43
Name: starttime, Length: 16738, dtype: datetime64[ns]


In [ ]:
#calculate duration of call
raw_cdr_data['endtime']=pd.to_datetime(call_time_fetcher(raw_cdr_data[13].tolist()))

In [ ]:
#difference between start and end time gives duration, convert that into minutes

raw_cdr_data['duration']=(raw_cdr_data['endtime']-raw_cdr_data['starttime']).astype('timedelta64[m]')

In [ ]:
raw_cdr_data['duration']

0        37.0
1        37.0
2         0.0
3        33.0
4         1.0
         ... 
16733     0.0
16734     0.0
16735     0.0
16736     0.0
16737     3.0
Name: duration, Length: 16738, dtype: float64

In [ ]:
raw_cdr_data['hourly_range']=hourly_range(raw_cdr_data['time'].tolist())
print(raw_cdr_data['hourly_range'])

0        032717:00 - 032717:59
1        032717:00 - 032717:59
2        052652:00 - 052652:59
3        052735:00 - 052735:59
4        060434:00 - 060434:59
                 ...          
16733    192334:00 - 192334:59
16734    192352:00 - 192352:59
16735    192352:00 - 192352:59
16736    192405:00 - 192405:59
16737    192143:00 - 192143:59
Name: hourly_range, Length: 16738, dtype: object


In [ ]:
raw_cdr_data['date']=date_modifier(raw_cdr_data['date'].tolist())

In [ ]:
raw_cdr_data['date']

0        2019-06-20
1        2019-06-20
2        2019-06-20
3        2019-06-20
4        2019-06-20
            ...    
16733    2019-06-25
16734    2019-06-25
16735    2019-06-25
16736    2019-06-25
16737    2019-06-25
Name: date, Length: 16738, dtype: object

In [ ]:
raw_cdr_data['weekly_range']=weekly_range(raw_cdr_data['date'].tolist())


In [ ]:
print(raw_cdr_data['weekly_range'])

0        Thursday
1        Thursday
2        Thursday
3        Thursday
4        Thursday
           ...   
16733     Tuesday
16734     Tuesday
16735     Tuesday
16736     Tuesday
16737     Tuesday
Name: weekly_range, Length: 16738, dtype: object


In [ ]:
raw_cdr_data.to_csv("cdr_data.csv",index=None) #do not store row index

* Client requirement: From cleaned CSV data create 3 csv files


*   Call data
*   device data
*   serice data





#Required features
"""
   column no -  actual name
   
    1            serviceProvider
    
    4            group Information
    5            direction
    9            startTime
    13           EndTime
    14           Miss Call Information
    31           Group ID
    120          userId
    180          twoStageDialingDigits
    146          relatedCallId
    147          relatedCallIdReason
    267          vpDialingfacResult
    312          locationType
    345          userAgent
  
                 date
                 starttime
                 endtime
                 duration
                 hourly_range
                 weekly_range
"""   


In [ ]:
dataset_name="cdr_data.csv"


In [ ]:
# low_memory=False as some columns contains mixed datatype
#header=None is used as dataset doesn't contain column name
call_dataset=pd.read_csv(dataset_name,usecols=call_columns,low_memory=False)

In [ ]:
 #Required columns
call_columns = ["4", "5","14", "31", "120", "147", "267", "312", "345", \
                "date","starttime", "endtime","duration", "hourly_range","weekly_range"]

# We have used low_memory = False as some columns contains mixed datatype
# header =  None is used as dataset doesn't contain column name
call_dataset = pd.read_csv(dataset_name, usecols = call_columns,low_memory = False)
    


# coulmns for service data
service_columns = ["31", "120", "147", "345","date", "starttime", "endtime","duration"]
service_dataset = call_dataset[service_columns]


# columns for device data
device_columns = ["5", "31", "120", "312", "345", "date","starttime", "endtime","duration"]
device_dataset = call_dataset[device_columns]



# Output


# Renaming columns name according to the Client

call_dataset = call_dataset.rename(columns = {"4":"Group", "5":"Call_Direction","14":"Missed Calls",
                                        "31":"GroupID", "120":"UserID", "147":"Features", "267":" vpDialingfacResult",
                                        "312":"UsageDeviceType",
                                        "345":"UserDeviceType"})


service_dataset = service_dataset.rename(columns={"120":"UserID", 
                                              "31":"GroupID", "147":"FeatureName",
                                              "345":"UserDeviceType","date":"FeatureEventDate"
                                              })



device_dataset = device_dataset.rename(columns={"5": "DeviceEventTypeDirection", 
                                  "120":"UserID", "31":"GroupID", 
                                  "345":"UserDeviceType","date":"DeviceEventDate", 
                                  "312":"UsageDeviceType"})




call_dataset.to_csv("Call_data.csv", index=None)
service_dataset.to_csv("Service_data.csv", index=None)
device_dataset.to_csv("Device_data.csv", index=None)
